In [24]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from astropy.table import Table, Column, join, unique
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
import pandas as pd
import numpy as np
import os
import string as s
import glob

In [25]:
#deine the path to save the plots
table_path = '/Users/Briana/Documents/Grad_School/HPS/Paper_tables'

# Build Master Dataframe

In [26]:
hps_prop = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_flux_prop_table.csv')
hps_prop.rename(columns={'ID_x':'ID', 'Detect':'Selection'}, inplace=True)
hps_prop.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0.1','ID_y' ], inplace=True)
#print hps_prop.loc[:,['HPS_name', 'Selection', 'ID', 'z', 'LogM', 'metallicity']]
print(hps_prop.columns)

print(hps_prop.loc[:,['LogM_eu','met_eu', 'SFR_met_corr_eu']])

Index([u'Selection', u'HPS_name', u'Hb_absorption', u'ID', u'Source',
       u'[Ha]6562', u'[Ha]6562_e', u'[Hb]4861', u'[Hb]4861_e', u'[NII]6583',
       u'[NII]6583_e', u'[NeIII]3870', u'[NeIII]3870_e', u'[Ne]3870',
       u'[OIII]5007', u'[OIII]5007_e', u'[OIII]_HPS', u'[OIII]_e_HPS',
       u'[OII]3727', u'[OII]3727_e', u'E(B-V)', u'E(B-V)_err', u'OIII_int_err',
       u'OIII_intrinsic', u'met_err', u'metallicity', u'z', u'LogM',
       u'LogM_eu', u'LogM_el', u'LogM_e', u'met_eu', u'met_el', u'E(B-V)_eu',
       u'E(B-V)_el', u'OII_corr', u'OII_corr_eu', u'OII_corr_el', u'Lum_dist',
       u'SFR', u'SFR_met_corr', u'SFR_met_corr_eu', u'SFR_met_corr_el',
       u'SFR_eu', u'SFR_el', u'log_SFR_met_corr', u'log_SFR_met_corr_el',
       u'log_SFR_met_corr_eu'],
      dtype='object')
    LogM_eu    met_eu  SFR_met_corr_eu
0     0.127  0.101926         2.846638
1     0.100  0.099669         6.138404
2     0.098  0.135156         0.884668
3     0.113  0.101564         1.331109
4     0.072

In [27]:
hps_orig = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_cat_table.dat')
hps_orig_sub = hps_orig.loc[:,['HPS_name', 'OII_RA', 'OII_DEC', 'OII_EW', 'OIII1_RA', 'OIII1_DEC', 'OIII1_EW']]

In [28]:
hps = pd.merge(hps_prop, hps_orig_sub, on='HPS_name', how='left')
hps.rename(columns={'OII_RA':'RA', 'OII_DEC':'DEC', 'OIII1_EW':'OIII_EW'}, inplace=True)
hps['RA'].iloc[27] = hps['OIII1_RA'].iloc[27]
hps['DEC'].iloc[27] = hps['OIII1_DEC'].iloc[27]
hps['RA'].iloc[28] = hps['OIII1_RA'].iloc[28]
hps['DEC'].iloc[28] = hps['OIII1_DEC'].iloc[28]
hps.drop(columns=['OIII1_RA', 'OIII1_DEC'], inplace=True)
print (hps.columns)
hps.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_all_props.csv')

Index([u'Selection', u'HPS_name', u'Hb_absorption', u'ID', u'Source',
       u'[Ha]6562', u'[Ha]6562_e', u'[Hb]4861', u'[Hb]4861_e', u'[NII]6583',
       u'[NII]6583_e', u'[NeIII]3870', u'[NeIII]3870_e', u'[Ne]3870',
       u'[OIII]5007', u'[OIII]5007_e', u'[OIII]_HPS', u'[OIII]_e_HPS',
       u'[OII]3727', u'[OII]3727_e', u'E(B-V)', u'E(B-V)_err', u'OIII_int_err',
       u'OIII_intrinsic', u'met_err', u'metallicity', u'z', u'LogM',
       u'LogM_eu', u'LogM_el', u'LogM_e', u'met_eu', u'met_el', u'E(B-V)_eu',
       u'E(B-V)_el', u'OII_corr', u'OII_corr_eu', u'OII_corr_el', u'Lum_dist',
       u'SFR', u'SFR_met_corr', u'SFR_met_corr_eu', u'SFR_met_corr_el',
       u'SFR_eu', u'SFR_el', u'log_SFR_met_corr', u'log_SFR_met_corr_el',
       u'log_SFR_met_corr_eu', u'RA', u'DEC', u'OII_EW', u'OIII_EW'],
      dtype='object')


In [29]:
print(hps[hps['Selection']=='[OIII]5007'].loc[:,['HPS_name', 'LogM', 'metallicity', '[OII]3727', '[OII]3727_e', '[OIII]5007', '[OIII]5007_e']])

            HPS_name   LogM  metallicity  [OII]3727  [OII]3727_e  [OIII]5007  \
27  HPS100021+021351  9.100     8.027626   1.012775     0.686858        2.44   
28  HPS123652+621125  8.162     7.655729   0.902217     0.482808        0.66   

    [OIII]5007_e  
27      0.274354  
28      0.206453  


In [30]:
phot = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/hps_table.dat', sep='\s+')
print(phot)

   field  obj_id       z    Hb_FLUX    Hb_ERR         f_B        e_B  \
0    HPS      65  0.1121   2.480000  0.364706   85.507157   8.550716   
1    HPS      35  0.0841  29.090000  0.309798    0.000000   0.000000   
2    HPS     105  0.1094   2.370000  0.353731    5.764362   0.576436   
3    HPS     138  0.1027   3.360000  0.270968   20.436104   2.043610   
4    HPS     219  0.1231   4.030000  0.281818   37.724717   3.772472   
5    HPS     225  0.1243   1.180000  0.173529   19.127132   1.912713   
6    HPS     278  0.1226   3.680000  0.201093   26.081510   2.608151   
7    HPS     326  0.1222   1.940000  0.269444   20.687192   2.068719   
8    HPS     375  0.0786  14.130000  0.374801  133.585180  13.358518   
9    HPS     386  0.0891   4.030000  0.213228   30.397075   3.039708   
10   HPS     413  0.1389   4.360000  0.171654   51.505809   5.150581   
11   HPS     449  0.0887   1.720000  0.215000   22.217765   2.221776   
12   HPS     458  0.1359   2.670000  0.160843   40.499639   4.04

# Build Object Table

In [31]:
hps_objs = hps.loc[:,['HPS_name', 'ID','Selection', 'z', 'RA', 'DEC']]
hps_objs.to_csv('/Users/Briana/Documents/Grad_School/HPS/Paper_tables/hps_objs_enhanced.csv')

In [32]:
#get table that lists the HPS fields by RA
hps_fields = pd.read_table('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hps_fields.txt', delimiter=None, header=50, skiprows=[52,53])

def build_astropy_coords(row):
    ra  = row['RAJ2000'].split()
    dec = row['DEJ2000'].split()
    c = SkyCoord(ra[0]+'h'+ra[1]+'m'+ra[2]+'s', dec[0]+'d'+dec[1]+'m'+dec[2]+'s', frame='icrs')
    return np.round(c.ra.degree,0)

hps_fields['RA_round'] = hps_fields.apply(build_astropy_coords, axis=1)
hps_fields = hps_fields.loc[:,['Field', 'RA_round']]

hps_objs['RA_round'] = np.round(hps_objs['RA'],0)
hps_objs = pd.merge(hps_objs, hps_fields, on='RA_round', how='left')
hps_objs = hps_objs.loc[:,['HPS_name', 'ID','Selection', 'z', 'RA', 'DEC', 'Field']]

/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [33]:
obj_table = Table.from_pandas(hps_objs)
obj_table['ID'] = obj_table['ID'].astype(int)

round_objs = ['RA', 'DEC']
for c in round_objs:
    obj_table[c].format = '7.6f'
obj_table['z'].format = '7.4f'

obj_table.sort(['ID'])

print (obj_table )   
obj_table.write(os.path.join(table_path, 'HPS_objs.tex'), format='ascii.aastex', overwrite=True)

    HPS_name      ID Selection     z        RA        DEC      Field  
---------------- --- ---------- ------- ---------- --------- ---------
HPS022127-043019  35  [OII]3727  0.0841  35.364375 -4.505306 XMM-LSS  
HPS030630+000128  44  [OII]3727  0.1191  46.628042  0.024667 MUNICS-S2
HPS030638+000015  65  [OII]3727  0.1121  46.661458 -0.004417 MUNICS-S2
HPS030638+000240  67  [OII]3727  0.1123  46.662125 -0.044472 MUNICS-S2
HPS030649+000314 105  [OII]3727  0.1094  46.707250 -0.054000 MUNICS-S2
HPS030651-000234 118  [OII]3727  0.1072  46.716417 -0.042972 MUNICS-S2
HPS030652+000123 119  [OII]3727  0.1121  46.716750  0.023167 MUNICS-S2
HPS030655-000050 125  [OII]3727  0.1118  46.729583 -0.013972 MUNICS-S2
HPS030655+000213 129  [OII]3727  0.1377  46.732542  0.037083 MUNICS-S2
HPS030657+000139 138  [OII]3727  0.1027  46.740125 -0.027528 MUNICS-S2
             ... ...        ...     ...        ...       ...       ...
HPS100037+021254 300  [OII]3727  0.1159 150.156333  2.215167 COSMOS   
HPS100

# Build Flux Table

In [34]:
#OII Fluxes

hps_fluxes = hps[hps['Selection']=='[OII]3727'].loc[:,['HPS_name', 'ID', 'Source', '[OII]3727', '[OII]3727_e', '[NeIII]3870', '[NeIII]3870_e', '[Hb]4861', '[Hb]4861_e', '[OIII]5007', '[OIII]5007_e', 'Hb_absorption']]
hps_fluxes['Hb_absorption_e'] = hps_fluxes['Hb_absorption']*0.3

hps_fluxes.to_csv('/Users/Briana/Documents/Grad_School/HPS/Paper_tables/hps_fluxes_OII_enhanced.csv')

flux_lis = ['[OII]3727', '[NeIII]3870', '[Hb]4861', '[OIII]5007', 'Hb_absorption']

for f in flux_lis:
    fstr = [(str("%.3f" % round(hps_fluxes[f].iloc[i],3))+r'$\pm$'+str("%.3f" % round(hps_fluxes[f+'_e'].iloc[i],3))) if ~np.isnan(hps_fluxes[f].iloc[i]) else r'\nodata'  for i in range(len(hps_fluxes))]
    hps_fluxes[f] = fstr
    
hps_fluxes = hps_fluxes.loc[:,['HPS_name', 'ID', 'Source', '[OII]3727', '[NeIII]3870', '[Hb]4861','[OIII]5007','Hb_absorption']]

flux_table = Table.from_pandas(hps_fluxes)
flux_table['ID'] = flux_table['ID'].astype(int)

flux_table.sort(['ID'])
    
print (flux_table)
flux_table.write(os.path.join(table_path, 'HPS_Fluxes_OII.tex'), format='ascii.aastex', overwrite=True)

    HPS_name      ID Source ...    [OIII]5007     Hb_absorption 
---------------- --- ------ ... ---------------- ---------------
HPS022127-043019  35    HPS ... 39.230$\pm$0.375 8.912$\pm$2.673
HPS030630+000128  44   LRS2 ...  3.055$\pm$0.273 0.664$\pm$0.199
HPS030638+000015  65    HPS ...  6.700$\pm$1.218 8.694$\pm$2.608
HPS030638+000240  67   LRS2 ...  0.073$\pm$1.008 1.450$\pm$0.435
HPS030649+000314 105    HPS ...  5.890$\pm$0.364 0.498$\pm$0.149
HPS030651-000234 118   LRS2 ...  1.663$\pm$0.323 0.680$\pm$0.204
HPS030652+000123 119   LRS2 ... 13.679$\pm$0.943 3.366$\pm$1.010
HPS030655-000050 125   LRS2 ... 14.999$\pm$2.330 7.120$\pm$2.136
HPS030655+000213 129   LRS2 ...  7.685$\pm$0.651 0.593$\pm$0.178
HPS030657+000139 138    HPS ...  5.140$\pm$0.221 1.895$\pm$0.568
             ... ...    ... ...              ...             ...
HPS100032+021356 278    HPS ... 15.240$\pm$0.230 2.086$\pm$0.626
HPS100037+021254 300   LRS2 ...  1.762$\pm$0.203 1.670$\pm$0.501
HPS100039+021246 303   LR

In [35]:
#OIII Fluxes

hps_fluxes = hps[hps['Selection']=='[OIII]5007'].loc[:,['HPS_name', 'ID', 'Source', '[OII]3727', '[OII]3727_e', '[NeIII]3870', '[NeIII]3870_e', '[Hb]4861', '[Hb]4861_e', '[OIII]5007', '[OIII]5007_e','[Ha]6562', '[Ha]6562_e', '[NII]6583', '[NII]6583_e', 'Hb_absorption']]
hps_fluxes['Hb_absorption_e'] = hps_fluxes['Hb_absorption']*0.3

hps_fluxes.to_csv('/Users/Briana/Documents/Grad_School/HPS/Paper_tables/hps_fluxes_OIII_enhanced.csv')

flux_lis = ['[OII]3727', '[NeIII]3870', '[Hb]4861', '[OIII]5007', '[Ha]6562', '[NII]6583', 'Hb_absorption']
for f in flux_lis:
    fstr = [(str("%.3f" % round(hps_fluxes[f].iloc[i],3))+r'$\pm$'+str("%.3f" % round(hps_fluxes[f+'_e'].iloc[i],3))) if ~np.isnan(hps_fluxes[f].iloc[i]) else r'\nodata'  for i in range(len(hps_fluxes))]
    hps_fluxes[f] = fstr
    
hps_fluxes = hps_fluxes.loc[:,['HPS_name', 'ID', 'Source', '[OII]3727', '[NeIII]3870', '[Hb]4861','[OIII]5007', '[Ha]6562', '[NII]6583','Hb_absorption']]

flux_table = Table.from_pandas(hps_fluxes)
flux_table['ID'] = flux_table['ID'].astype(int)

flux_table.sort(['ID'])
    
print (flux_table)
flux_table.write(os.path.join(table_path, 'HPS_Fluxes_OIII.tex'), format='ascii.aastex', overwrite=True)

    HPS_name      ID Source ...     [Ha]6562       [NII]6583     Hb_absorption 
---------------- --- ------ ... --------------- --------------- ---------------
HPS100021+021351 234   LRS2 ... 1.428$\pm$0.152 0.074$\pm$0.058 0.120$\pm$0.036
HPS123652+621125 430   LRS2 ... 1.208$\pm$0.127 0.015$\pm$0.044 0.300$\pm$0.090


# Build LRS2 Observation Table

In [36]:
#lines = pd.read_csv('/Volumes/Bri_USB/hps_lrs2_fits_wNe_4.csv')
lines = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hps_lrs2_fits_wNe_4.csv')
OIII_lines = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hps_lsr2_fits_wNe_OIIIobs.csv')

In [37]:
#remove row 35 because no signal and row 25, 37 because bad data
lines = lines[lines['HPS_name'] != 'HPS123652+621125']
lines = lines[lines['HPS_name'] != 'HPS100021+021351']
lines = lines.drop([37], axis=0)
lines = lines.loc[:, ~lines.columns.str.contains('^Unnamed')]
OIII_lines = OIII_lines.loc[:, ~OIII_lines.columns.str.contains('^Unnamed')]
lines['Selection'] = 'OII'
OIII_lines['Selection'] = 'OIII'
lines = lines.append(OIII_lines, sort=True)
lines = lines.reset_index()

obs_lis = lines['Obs_ID'].str.split(pat = "_", expand=True)
lines['Date'] = obs_lis.iloc[:,0]
lines['obs'] = obs_lis.iloc[:,1].str.split(pat='s', expand=True).iloc[:,1]
lines['exp'] = obs_lis.iloc[:,2]
lines['exptime'] = None

In [38]:
for i in range(len(lines)):
    if lines['Selection'].iloc[i] == 'OII':
        fil = glob.glob('/Volumes/BRIANA_3T/external_backup_3/HPS/LRS2_reduction_greg/reductions3/'+str(lines['Date'].iloc[i])+'/lrs2/lrs2*'+str(lines['obs'].iloc[i])+'/'+lines['exp'].iloc[i]+'/lrs2/spectrum_uv.fits')
        dat, hdr = fits.getdata(fil[0], 0, header=True)
        exptime = hdr['EXPTIME']
        lines.at[i, 'exptime'] = exptime
        
#group the observations by name and date - add the expsoure times 
lines = lines.loc[:,['HPS_name', 'Date','obs', 'exp', 'exptime']]
line_groups = pd.DataFrame({'Exposure Time' : lines.groupby( [ "HPS_name", "Date"] )["exptime"].sum()}).reset_index()
line_groups['Instrument'] = 'LRS2-Blue'
line_groups = line_groups.round({'Exposure Time': 2})

#manually add row for the OIII emitter observations no included 
line_groups.loc[13, 'Exposure Time'] = 1615.91
line_groups.loc[20, 'Exposure Time'] = 1815.59
line_groups = line_groups.append({'HPS_name':'HPS100021+021351', 'Date':20190126, 'Exposure Time':1816.91 ,'Instrument':'LRS2-Red'}, ignore_index=True)
line_groups = line_groups.append({'HPS_name':'HPS123652+621125', 'Date':20190202, 'Exposure Time':1817.14 ,'Instrument':'LRS2-Red'}, ignore_index=True)
line_groups = line_groups.append({'HPS_name':'HPS123652+621125', 'Date':20190316, 'Exposure Time':2416.51+2416.92+2415.97 ,'Instrument':'LRS2-Red'}, ignore_index=True)

line_groups.sort_values(by=['HPS_name', 'Date'], inplace=True)

IndexError: list index out of range

In [39]:
hps_ids = hps.loc[:,['HPS_name', 'ID']]
lrs2_dates = pd.merge(line_groups, hps_ids, on='HPS_name', how='left')
lrs2_dates = lrs2_dates.loc[:,['HPS_name', 'ID', 'Date', 'Exposure Time', 'Instrument']]

new_inst = [lrs2_dates['Instrument'].iloc[i][0:6] for i in range(len(lrs2_dates))]
lrs2_dates['Instrument'] = new_inst

lrs2_table = Table.from_pandas(lrs2_dates)
lrs2_table['ID'] = lrs2_table['ID'].astype(int)

lrs2_table.sort(['ID'])
lrs2_table['Exposure Time'].format = '7.2f'

print (lrs2_table)
lrs2_table.write(os.path.join(table_path, 'HPS_lrs2_obs.tex'), format='ascii.aastex', overwrite=True)

NameError: name 'line_groups' is not defined

# Build HPS Properties Table

In [40]:
print(hps.loc[:, ['LogM_eu', 'met_eu', 'E(B-V)_eu','log_SFR_met_corr_eu']])

    LogM_eu    met_eu  E(B-V)_eu  log_SFR_met_corr_eu
0     0.127  0.101926   0.153066             0.264991
1     0.100  0.099669   0.147990             0.338250
2     0.098  0.135156   0.161233             0.524529
3     0.113  0.101564   0.154612             0.350625
4     0.072  0.102356   0.151050             0.331218
5     0.076  0.107515   0.156370             0.389381
6     0.090  0.135444   0.159118             0.566158
7     0.075  0.106403   0.152583             0.356241
8     0.071  0.097483   0.148832             0.307604
9     0.085  0.103369   0.149293             0.350573
10    0.075  0.100742   0.154731             0.311384
11    0.069  0.115013   0.155357             0.428310
12    0.067  0.091937   0.148908             0.254171
13    0.108  0.113949   0.158125             0.446275
14    0.118  0.163672   0.170770             0.280960
15    0.124  0.120615   0.163691             0.399362
16    0.108  0.118425   0.154295             0.410852
17    0.102  0.169193   0.16

In [41]:
hps_props = hps.loc[:,['HPS_name', 'ID', 'LogM', 'metallicity', 'E(B-V)', 'log_SFR_met_corr', 'LogM_eu', 'LogM_el', 'met_eu', 'met_el', 'E(B-V)_eu', 'E(B-V)_el', 'log_SFR_met_corr_eu', 'log_SFR_met_corr_el']]
hps_props.rename(columns={'log_SFR_met_corr':'LogSFR', 'log_SFR_met_corr_eu':'LogSFR_eu', 'log_SFR_met_corr_el':'LogSFR_el', 'met_eu':'metallicity_eu', 'met_el':'metallicity_el'}, inplace=True)

print hps_props
hps_props.to_csv('/Users/Briana/Documents/Grad_School/HPS/Paper_tables/hps_props_enhanced.csv')

prop_lis = ['LogM', 'metallicity', 'E(B-V)', 'LogSFR']
for p in prop_lis:
    hps_props[p]=['$'+str("%.2f" % np.round(hps_props[p].iloc[i],2))+'^{+'+str("%.2f" % np.round(hps_props[p+'_eu'].iloc[i],3))+'}_{-'+str("%.2f" % np.round(hps_props[p+'_el'].iloc[i],3))+'}$'  for i in range(len(hps_props))]

hps_props = hps_props.loc[:,['HPS_name', 'ID', 'LogM', 'metallicity', 'E(B-V)', 'LogSFR']]
    
prop_table = Table.from_pandas(hps_props)
prop_table['ID'] = prop_table['ID'].astype(int)

prop_table.sort(['ID'])
    
print(prop_table)
prop_table.write(os.path.join(table_path, 'HPS_properties.tex'), format='ascii.aastex', overwrite=True)



            HPS_name     ID    LogM  metallicity    E(B-V)    LogSFR  LogM_eu  \
0   HPS030638+000015   65.0  10.800     8.891323  0.342487  0.668592    0.127   
1   HPS022127-043019   35.0   9.989     8.773487  0.395548  0.896308    0.100   
2   HPS030649+000314  105.0   8.712     8.565194  0.396673 -0.135499    0.098   
3   HPS030657+000139  138.0   9.472     8.775848  0.410992  0.216860    0.113   
4   HPS100018+021818  219.0  10.374     8.792618  0.334123  0.515328    0.072   
5   HPS100018+021426  225.0   9.420     8.727978  0.381651  0.048613    0.076   
6   HPS100032+021356  278.0   9.472     8.497223  0.393656  0.242905    0.090   
7   HPS100045+021823  326.0   9.554     8.762508  0.363982  0.162735    0.075   
8   HPS123636+621135  375.0   9.799     8.820106  0.415997  0.659808    0.071   
9   HPS123641+621131  386.0   9.234     8.760042  0.358003  0.159228    0.085   
10  HPS123648+621426  413.0   9.902     8.830266  0.345013  0.718518    0.075   
11  HPS123659+621404  449.0 